In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Tentukan jalur direktori untuk data latih, validasi, dan uji
train_data = "../Oprek/Ori_Data_Aug/train" 
val_data = "../Oprek/Ori_Data_Aug/val"
test_data = "../Oprek/Ori_Data_Aug/test"

# train_data = '../Oprek/data_split/train'
# test_data = '../Oprek/data_split/test'

In [3]:
# Applying Data normalization and Data augmentation
img_datagen = ImageDataGenerator(
    # rescale=1./255
)

# validation_datagen = ImageDataGenerator(
#     # rescale=1./255
# )

train_generator = img_datagen.flow_from_directory(
    train_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(224, 224),
    class_mode="categorical"
)

validation_generator = img_datagen.flow_from_directory(
    val_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(224, 224),
    class_mode="categorical"
)

test_generator = img_datagen.flow_from_directory(
    test_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(224, 224),
    class_mode="categorical"
)

Found 8400 images belonging to 20 classes.
Found 2400 images belonging to 20 classes.
Found 1200 images belonging to 20 classes.


# Model CNN dengan Tuning Parameter dan Dropout(0.75)

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers

# Membuat model CNN dengan Batch Normalization, Regularization, dan Dropout
model = models.Sequential()

# Blok 1
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.75))  # Menambahkan layer Dropout

# Blok 2
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.75))  

# Blok 3
model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.75))  

# Blok 4
model.add(layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.75))

model.add(layers.Flatten())

# Dense Layer 1
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.75))

# Dense Layer 2
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.75))

model.add(layers.Dense(20, activation='softmax'))

# Compile model dengan pengaturan tambahan
opt = optimizers.Adam(learning_rate=0.001, clipvalue=0.5)  # Gradient clipping
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print ringkasan model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 64)      2

In [5]:
# Fungsi untuk menetapkan learning rate secara dinamis
def lr_scheduler(epoch, lr):
    if epoch % 30 == 0 and epoch > 0:
        lr = lr * 0.9  # Contoh: Mengurangi learning rate setiap 10 epoch
    return lr

# Fungsi untuk menghentikan pelatihan jika akurasi mencapai 95%
class AccuracyThreshold(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.98:
            print("\nAkurasi mencapai 95%! Menghentikan pelatihan.")
            self.model.stop_training = True

In [6]:
# Menambahkan learning rate scheduler dan early stopping
lr_schedule = LearningRateScheduler(lr_scheduler)
accuracy_threshold = AccuracyThreshold()
callbacks_list = [lr_schedule, accuracy_threshold]

In [ ]:
# Train the model with EarlyStopping callback
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks_list  # Menggunakan callbacks_list yang sudah termasuk EarlyStopping
)

Epoch 1/50
  1/263 [..............................] - ETA: 5:20:24 - loss: 4.4995 - accuracy: 0.0312

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2) 
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

In [ ]:
# Evaluasi model pada data uji
evaluation_result = model.evaluate(test_generator)

# Menampilkan hasil evaluasi
print(f"Test Loss: {evaluation_result[0]}")
print(f"Test Accuracy: {evaluation_result[1]}")

# Testing

In [ ]:
# Menyimpan model setelah melatih
model.save('../Oprek/model_v7.h5')

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Mengasumsikan Anda memiliki model yang telah dilatih
model = tf.keras.models.load_model('../Oprek/model_v7.h5')

# Mengasumsikan Anda memiliki gambar yang akan diprediksi
path_gambar = '../Oprek/Ori_byPhiard/prediction/ba17.png'

# Memuat dan memproses gambar
img = tf.keras.preprocessing.image.load_img(path_gambar, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Membuat batch

# Memprediksi probabilitas kelas
prediksi = model.predict(img_array)
kelas_terprediksi = tf.argmax(prediksi, axis=1).numpy()[0]

# Mendefinisikan label kelas
label_kelas = ['ba', 'ca', 'da', 'dha', 'ga', 'ha', 'ja', 'ka', 'la', 'ma',
               'na', 'nga', 'nya', 'pa', 'ra', 'sa', 'ta', 'tha', 'wa', 'ya']

# Mencetak label kelas yang diprediksi
print("Kelas yang diprediksi:", label_kelas[kelas_terprediksi])

# Menampilkan gambar
plt.imshow(img)
plt.title(f'Predicted Class: {label_kelas[kelas_terprediksi]}')
plt.show()

In [ ]:
# import os  
# import matplotlib.pyplot as plt
# from keras.preprocessing import image
# from keras.models import load_model
# import numpy as np

# # Menyimpan model setelah melatih
# # model.save('../Oprek/model_v7.h5')

# # Memuat model dari file
# # loaded_model = load_model('model_v7a.h5')

# # Memuat gambar
# img_path = '../Oprek/Ori_byPhiard/prediction/ba17.png'
# img = image.load_img(img_path,
#                      target_size=(224, 224),
#                      color_mode='grayscale')
# img = image.img_to_array(img)
# img = np.expand_dims(img, axis=0)

# # Memuat model menggunakan path absolut (sebaiknya menggunakan path yang benar)
# model_path = '../Oprek/model_v7.h5'
# saved_model = load_model(model_path)
# output = saved_model.predict(img)

# # Bagian sisanya dari kode Anda tetap tidak berubah
# max_value = np.max(output)
# pos = np.argmax(output)

# max = output[0][0]
# pos = 0
# for i in range(1, 19): 
#     if output[0][i] > max: 
#         max = output[0][i]
#         pos = i
         
# print(output)
# print(max)


# if (pos == 0) :
#     print("ba")
# elif (pos == 1) :
#     print('ca')
# elif (pos == 2) :
#     print('da')
# elif (pos == 3) :
#     print('dha')
# elif (pos == 4) :
#     print('ga')
# elif (pos == 5) :
#     print('ha')
# elif (pos == 6) :
#     print('ja')
# elif (pos == 7) :
#     print('ka')
# elif (pos == 8) :
#     print('la')
# elif (pos == 9) :
#     print('ma')
# elif (pos == 10) :
#     print('na')
# elif (pos == 11) :
#     print('nga')
# elif (pos == 12) :
#     print('nya')
# elif (pos == 13) :
#     print('pa')
# elif (pos == 14) :
#     print('ra')
# elif (pos == 15) :
#     print('sa')
# elif (pos == 16) :
#     print('ta')
# elif (pos == 17) :
#     print('tha')
# elif (pos == 18) :
#     print('wa')
# elif (pos == 19) :
#     print('ya')